# Preparing Training Dataset

In [ ]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner

# Data Preprocessing

In [ ]:
# perform stemming , remove stopwords , ....
# here it is not needed and hence I will not be performing these operations in here

# Converting Words to Embeddings

In [ ]:
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


dog : 1 represent the index 1 of the OHE of dog will be 1 and rest all indexes will be zeroes, similarly for others

In [ ]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}
# Assign each tag with a unique index
# Usually in real life you will encounter 32 possible POS tags but we will only consider 3 for now & keep them small, so we can see how the weights change as we train.


# Creating Model



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # input layer
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # hidden layer - The LSTM takes word embeddings as inputs, and outputs hidden states with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Training the Model

In [ ]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
# See what the scores are before training. Note that element i,j of the output is the score for tag j for word i. Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
  inputs = prepare_sequence(training_data[0][0], word_to_ix)
  tag_scores = model(inputs)
  print(tag_scores)

tensor([[-1.0500, -0.9256, -1.3714],
        [-1.0188, -0.9739, -1.3419],
        [-1.1330, -0.9662, -1.2126],
        [-1.1818, -0.9763, -1.1501],
        [-1.0766, -0.9916, -1.2439]])


In [ ]:
for epoch in range(300):  # normally you would NOT do 300 epochs, it is toy data
  for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Predicting using Model

In [ ]:
# See what the scores are after training
with torch.no_grad():
  inputs = prepare_sequence(training_data[0][0], word_to_ix)
  tag_scores = model(inputs)

  # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j for word i. The predicted tag is the maximum scoring tag.Here, we can see the predicted sequence below is 0 1 2 0 1
  # since 0 is index of the maximum value of row 1,1 is the index of maximum value of row 2, etc.Which is DET NOUN VERB DET NOUN, the correct sequence!
  print(tag_scores)

tensor([[-0.3892, -1.2426, -3.3890],
        [-2.1082, -0.1328, -5.8464],
        [-3.0852, -5.9469, -0.0495],
        [-0.0499, -3.4414, -4.0961],
        [-2.4540, -0.0929, -5.8799]])
